## Classe de fusão

Pega o vetor 1D, repete ele na grade 8x8 e concatena com o vídeo

# 1 Importanto bibliotecas

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

#Controlar a geração de números aleatórios para que os mesmos sempre sejam gerados
torch.manual_seed(0)
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cpu'

# 2 => Fusão

Cálculo da fusão dos componentes


In [ ]:
class Fusion(nn.Module):
    def __init__(self, h=8, w=8):
        super().__init__()
        self.h = h
        self.w = w

    def forward(self, z_v, z_a):
        if z_v.dim() != 4:
            raise ValueError(f"z_v precisa ser 4D, veio {z_v.shape}")
        if z_a.dim() != 2:
            raise ValueError(f"z_a precisa ser 2D, veio {z_a.shape}")

        #Garante a ordem do z_v em NCHW
        if z_v.shape[1] == self.h and z_v.shape[2] == self.w:
            #Permutar para
            print("Permutando")
            z_v = z_v.permute(0, 3, 1, 2).contiguous()
        elif z_v.shape[2] == self.h and z_v.shape[3] == self.w:
            #Mantém a mesma ordem
            print("Não faz nada")
            pass
        else:
            raise ValueError(f"Formato inesperado para z_v: {z_v.shape}")

        #-1 para n mudar a dimensão
        z_a_map = z_a[:, :, None, None].expand(-1, -1, self.h, self.w)
        #Juntar eixos [B,24,8,8]
        return torch.cat([z_v, z_a_map], dim=1)


## 3 => Difusion

Calculo da difusão para tentar validar se a fusão deu certo ou não.

In [ ]:
class Defusion(nn.Module):
    def __init__(self, c_vis=8, c_act=16):
        super().__init__()
        c_fused = c_vis + c_act

        #Conversões para visual e action
        self.to_visual = nn.Conv2d(c_fused, c_vis, kernel_size=1)

        self.to_action = nn.Sequential(
            nn.AdaptiveAvgPool2d((1, 1)),
            nn.Flatten(),
            nn.Linear(c_fused, 32),
            nn.SiLU(),
            nn.Linear(32, c_act)
        )

    def forward(self, z_fused):
        z_v_hat = self.to_visual(z_fused)
        z_a_hat = self.to_action(z_fused)
        return z_v_hat, z_a_hat


# 4 => Simulação + Checagem

Simulando os parametros dos dados + execução

In [11]:
batch_size = 32
z_v = torch.randn(batch_size, 8, 8, 8, device=device)
z_a = torch.randn(batch_size, 16, device=device)

fuser = Fusion().to(device)
defuser = Defusion().to(device)

z_out = fuser(z_v, z_a)
z_v_hat, z_a_hat = defuser(z_out)

print(f"Visual Shape: {z_v.shape}")
print(f"Action Shape: {z_a.shape}")
print(f"Fused Shape:  {z_out.shape}")
print(f"Visual_hat:   {z_v_hat.shape}")
print(f"Action_hat:   {z_a_hat.shape}")


Permutando
Visual Shape: torch.Size([32, 8, 8, 8])
Action Shape: torch.Size([32, 16])
Fused Shape:  torch.Size([32, 24, 8, 8])
Visual_hat:   torch.Size([32, 8, 8, 8])
Action_hat:   torch.Size([32, 16])


## 5 => Loss

A perda está sendo calculada em 200 steps e para cada step de 50 o resultado está sendo printado para comprarmos a evolução da loss.
Se diminuir então está bom. 

In [ ]:
opt = torch.optim.AdamW(list(defuser.parameters()), lr=1e-3)

for step in range(200):
    z_v = torch.randn(batch_size, 8, 8, 8, device=device)
    z_a = torch.randn(batch_size, 16, device=device)

    z_out = fuser(z_v, z_a)
    z_v_hat, z_a_hat = defuser(z_out)

    z_v_nchw = z_v.permute(0, 3, 1, 2).contiguous()

    #Calcula a loss de Visual e de action
    loss_v = F.mse_loss(z_v_hat, z_v_nchw)
    loss_a = F.mse_loss(z_a_hat, z_a)
    loss = loss_v + loss_a
    #Soma as duas e faz o comparativo

    opt.zero_grad()
    loss.backward()
    opt.step()

    if step % 50 == 0:
        print(step, float(loss), float(loss_v), float(loss_a))


C:\Users\Yuri\AppData\Local\Temp\ipykernel_29108\133901350.py:21: UserWarning: Converting a tensor with requires_grad=True to a scalar may lead to unexpected behavior.
Consider using tensor.detach() first. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\pytorch\torch\csrc\autograd\generated\python_variable_methods.cpp:837.)
  print(step, float(loss), float(loss_v), float(loss_a))


0 2.290241241455078 1.3401063680648804 0.9501349925994873
50 1.9029179811477661 1.085376501083374 0.8175414800643921
100 1.488761067390442 0.9056804180145264 0.5830806493759155
150 1.1687400341033936 0.7710345983505249 0.39770543575286865
